In [1]:
from monai.transforms import *
from monai.data import load_decathlon_datalist, Dataset
import torch
from torchvision.utils import make_grid

import gradio as gr

import pickle
demo = None

/home/sergio/anaconda3/envs/py39/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Failed to load image Python extension: '/home/sergio/anaconda3/envs/py39/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?


2024-02-10 15:58:14,281 - HTTP Request: GET https://api.gradio.app/gradio-messaging/en "HTTP/1.1 200 OK"


In [2]:
dl = load_decathlon_datalist("atlas_upd.json",
                             data_list_key = 'training_t1',
                             base_dir='/home/sergio/fastdrive/data/upd_preprocessed/atlas/'
                            )
transforms = Compose([LoadImaged(keys=['image','label'],image_only=True,ensure_channel_first=True),
                      ScaleIntensityRangePercentilesd(keys='image',lower=0,upper=98,b_min=0,b_max=1,clip=True),
                      CropForegroundd(keys=['image','label'],source_key='image',),
                      # Makes z as first dimension: (z,1,x,y)
                      Lambdad(keys=['image','label'],func=lambda x: x.movedim(-1,0)) 
                     ])

ds = Dataset(dl,transform=transforms,)

with open('atlas_argsort_anom.pickle','rb') as f:
    argsort_anom = pickle.load(f)
    
with open('atlas_flags.pickle','rb') as f:
    flags_options = pickle.load(f)

In [3]:
print("images with artefacts:",len([k for k,v in flags_options.items() if 'artefact' in v]))
print("images with skull:",len([k for k,v in flags_options.items() if 'skull-stripping' in v]))

images with artefacts: 24
images with skull: 74


In [10]:
# Load all images
options = {ds.data[i]['image'].split("/")[-1]:i.item() for i in argsort_anom}


# Uncomment to restrict dropdowns to either images with artefacts or skull-stripping issues
# options = {ds.data[i]['image'].split("/")[-1]:i.item() for i in argsort_anom if 'artefact' in flags_options[ds.data[i]['image'].split("/")[-1]]}
# options = {ds.data[i]['image'].split("/")[-1]:i.item() for i in argsort_anom if 'skull-stripping' in flags_options[ds.data[i]['image'].split("/")[-1]]}


In [11]:

keys_plot = ['image','label',]


def plot(zslice,xslice,yslice,):
    
    list_to_plot = []
    for k in keys_plot:
        for d,sl in zip([0,1,2],[zslice,xslice,yslice,]):
            img = imgs[k][:,0].movedim(d,0)[int(sl)].clone()
            
            if d==0:
                img[xslice] = 1
                img[:,yslice] = 1
                
            if d==1:
                img[zslice] = 1
                img[:,yslice] = 1
                
            if d==2:
                img[zslice] = 1
                img[:,xslice] = 1
           
            img = img.flip(0).clip(0.,1.)[None]
            list_to_plot.append(img)
    
    
    spatial_size = torch.stack([torch.tensor(i.shape[1:]) for i in list_to_plot],0).max(0)[0]
    pad = ResizeWithPadOrCrop(spatial_size=spatial_size,mode='constant')
    list_to_plot = [pad(i) for i in list_to_plot]
    
    return make_grid(list_to_plot,nrow=3).movedim(0,-1).numpy()
    
def plot_drop(name,zslice,xslice,yslice,):
    global imgs, flags_options
    idx = options[name]
    imgs = ds[idx]
    
    img_shape = imgs['image'].shape
    
    xsl = gr.Slider(0,img_shape[2]-1,value=min(xslice,img_shape[2]))
    ysl = gr.Slider(0,img_shape[3]-1,value=min(yslice,img_shape[3]))
    zsl = gr.Slider(0,img_shape[0]-1,value=min(zslice,img_shape[0]))
    
    if name not in flags_options.keys():
        flags_options[name] = []
    
    flags = gr.CheckboxGroup(choices=['artefact','skull-stripping','background'],value=flags_options[name])

    return plot(zslice,xslice,yslice,),zsl,xsl, ysl,flags  

def checkbox_input(name,flags):
    global flags_options
    flags_options[name] = flags


if demo:
    demo.close()
    
with gr.Blocks() as demo:
    # gr.Markdown("**Select** an image from dropdown to see the output.")
    with gr.Row():
        with gr.Column(scale=1):
            with gr.Row():
                inp = gr.Dropdown(label="Image",choices=options.keys())
            
            with gr.Row():
                coords = (
                    gr.Slider(0,100,value=50,label="z"),
                    gr.Slider(0,100,value=50,label="X"),
                    gr.Slider(0,100,value=50,label="y"),
                         )
            with gr.Row():
                flags = gr.CheckboxGroup(label='Flags', choices=['artefact','skull-stripping','background'])

       
        with gr.Column(scale=6):
            outputs =gr.Image(height=600, width=900)
    
    inp.change(fn=plot_drop,inputs=[inp,*coords],outputs=[outputs, *coords, flags])
    [sl.change(fn=plot,inputs=[*coords,], outputs=outputs) for sl in coords]
    flags.change(fn=checkbox_input,inputs=[inp,flags])
    
    
demo.launch(height=600,
            width=900,
            server_port=7860,
            server_name="0.0.0.0")

Closing server running on port: 7860
Running on local URL:  http://0.0.0.0:7860
2024-02-10 15:59:55,096 - HTTP Request: GET http://localhost:7860/startup-events "HTTP/1.1 200 OK"
2024-02-10 15:59:55,099 - HTTP Request: HEAD http://localhost:7860/ "HTTP/1.1 200 OK"

To create a public link, set `share=True` in `launch()`.


2024-02-10 15:59:55,679 - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2024-02-10 15:59:55,811 - HTTP Request: POST https://api.gradio.app/gradio-initiated-analytics/ "HTTP/1.1 200 OK"
2024-02-10 15:59:55,871 - HTTP Request: POST https://api.gradio.app/gradio-launched-telemetry/ "HTTP/1.1 200 OK"


In [9]:
demo.close()

In [12]:
# images with artefacts:",
[k for k,v in flags_options.items() if 'artefact' in v]


['sub-r005s058_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s044_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s115_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s067_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s005_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r038s043_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s039_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s096_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s050_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s001_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r028s026_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r048s015_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s061_ses-1_space-MNI152NLin2009aSym_T1w_st

In [13]:
# Images with skull:,
[k for k,v in flags_options.items() if 'skull-stripping' in v]

['sub-r005s058_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s044_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r038s024_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s115_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r048s002_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s067_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r047s026_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s005_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r038s043_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s039_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r009s096_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r038s061_ses-1_space-MNI152NLin2009aSym_T1w_stripped_registered.nii.gz',
 'sub-r038s016_ses-1_space-MNI152NLin2009aSym_T1w_st